In [6]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate
from ml_logic.preprocessor import data_balancing, create_augmented_model
from ml_logic.preprocessor import load_and_preprocess_image


In [2]:
train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [4]:
# X_train = train.drop(columns='Disease_Risk')
X_train = data_balancing(table_link='../data/')
y_train = train['Disease_Risk']
X_eval  = eval.drop(columns='Disease_Risk')
y_eval = eval['Disease_Risk']


In [5]:
image_folder = '../data/train_images'
images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])
# images = np.array([load_and_preprocess_image(X_train.iloc[0].ID, image_folder)])
images.shape


(802, 224, 224, 3)

In [ ]:
eval_image_folder = '../data/eval_images'
eval_images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_eval.index])


In [ ]:
image_input = Input(shape=(224, 224, 3))

create_augmented_model
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)


#  NOTE: row_input would cause data leakage so it's been commented out.

# row_input = Input(shape=(X_train.shape[1],))
# y = Dense(64, activation='relu')(row_input)

# combined = concatenate([x, y])

z = Dense(12, activation='relu')(x)
z = Dense(64, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)


model = Model(inputs=image_input, outputs=z)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 394272)            0         
                                                                 
 dense_7 (Dense)             (None, 12)                4731276   
                                                                 
 dense_8 (Dense)             (None, 64)                832       
                                                           

In [ ]:
model.fit(
    images, y_train,
    validation_data=(eval_images, y_eval),
    epochs=10,
    batch_size=32
)


Epoch 1/10
60/60 [==============================] - 12s 194ms/step - loss: 0.5187 - accuracy: 0.7714 - val_loss: 0.5269 - val_accuracy: 0.7906
Epoch 2/10
60/60 [==============================] - 10s 174ms/step - loss: 0.4538 - accuracy: 0.7911 - val_loss: 0.6026 - val_accuracy: 0.7906
Epoch 3/10
60/60 [==============================] - 11s 175ms/step - loss: 0.4348 - accuracy: 0.8047 - val_loss: 0.6571 - val_accuracy: 0.7906
Epoch 4/10
60/60 [==============================] - 10s 173ms/step - loss: 0.4348 - accuracy: 0.8234 - val_loss: 0.5397 - val_accuracy: 0.7906
Epoch 5/10
60/60 [==============================] - 10s 169ms/step - loss: 0.4219 - accuracy: 0.8250 - val_loss: 0.5902 - val_accuracy: 0.7906
Epoch 6/10
60/60 [==============================] - 11s 182ms/step - loss: 0.4019 - accuracy: 0.8349 - val_loss: 0.5851 - val_accuracy: 0.7875
Epoch 7/10
60/60 [==============================] - 11s 183ms/step - loss: 0.3984 - accuracy: 0.8365 - val_loss: 0.5513 - val_accuracy: 0.7906

In [ ]:
model.evaluate(eval_images, y_eval)


20/20 [==============================] - 1s 51ms/step - loss: 0.6839 - accuracy: 0.7797


[0.6838960647583008, 0.7796875238418579]